# Text generation using LSTMs


This project demonstrates how to generate text using a character-based LSTM. We will work with a dataset of Shakespeare's writing from Andrej Karpathy's [The Unreasonable Effectiveness of Recurrent Neural Networks](http://karpathy.github.io/2015/05/21/rnn-effectiveness/). Given a sequence of characters from this data ("Shakespear"), train a model to predict the next character in the sequence ("e"). Longer sequences of text can be generated by calling the model repeatedly.

## Setup

**Get the google thing done**

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
import os
os.chdir('/content/drive/My Drive/NLP/text_generation/shakespeare')

### Import Keras and other libraries

In [0]:
import glob

from sklearn.utils import shuffle
import numpy as np

from keras.preprocessing.text import Tokenizer
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dense, Dropout, Embedding, Masking, Bidirectional
from keras.optimizers import Adam
from keras import backend

Using TensorFlow backend.


### Load the Shakespeare dataset

Run this code on your own data.

In [0]:
shakelist = glob.glob("./data/*.txt")

### Read the data

Reading every file

In [0]:
codetext = []
bookranges = []
for shakefile in shakelist:
    shaketext = open(shakefile, "r")
    start = len(codetext)
    codetext.append(shaketext.read())
    end = len(codetext)
    bookranges.append({"start": start, "end": end, "name": shakefile.rsplit("/", 1)[-1]})
    shaketext.close()

## Process the text

In [0]:
tokenizer = Tokenizer(lower=True, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n')
tokenizer.fit_on_texts(codetext)

### Vectorize the text

Before training, we need to map strings to a numerical representation. Create two lookup tables: one mapping words to numbers, and another for numbers to words.

In [0]:
word_idx = tokenizer.word_index
idx_word = tokenizer.index_word

Get the word counts

In [0]:
word_counts = tokenizer.word_counts
num_words = len(word_idx) + 1

Convert text to sequence of numbers

In [0]:
sequences = tokenizer.texts_to_sequences(codetext)

### Generate Features and Labels

In [0]:
features = []
labels = []

training_length = 50
# Iterate through the sequences of tokens
for seq in sequences:
    # Create multiple training examples from each sequence
    for i in range(training_length, training_length+300):
        # Extract the features and label
        extract = seq[i - training_length: i - training_length + 20]

        # Set the features and label
        features.append(extract[:-1])
        labels.append(extract[-1])

### The prediction task

Given a character, or a sequence of characters, what is the most probable next character? This is the task we're training the model to perform. The input to the model will be a sequence of characters, and we train the model to predict the output—the following character at each time step.

Since RNNs maintain an internal state that depends on the previously seen elements, given all the characters computed until this moment, what is the next character?


### Generate training and testing data

In [0]:
from sklearn.utils import shuffle
import numpy as np

features, labels = shuffle(features, labels, random_state=1)

# Decide on number of samples for training
train_end = int(0.7 * len(labels))

train_features = np.array(features[:train_end])
valid_features = np.array(features[train_end:])

train_labels = labels[:train_end]
valid_labels = labels[train_end:]

# Convert to arrays
X_train, X_valid = np.array(train_features), np.array(valid_features)

# Using int8 for memory savings
y_train = np.zeros((len(train_labels), num_words), dtype=np.int8)
y_valid = np.zeros((len(valid_labels), num_words), dtype=np.int8)

# One hot encoding of labels
for example_index, word_index in enumerate(train_labels):
    y_train[example_index, word_index] = 1

for example_index, word_index in enumerate(valid_labels):
    y_valid[example_index, word_index] = 1

This is just to check the features and labels

In [0]:
for i, sequence in enumerate(X_train[:2]):
    text = []
#     print(i, sequence)
    for idx in sequence:
        text.append(idx_word[idx])
        
    print('Features: ' + ' '.join(text)+'\n')
    print('Label: ' + idx_word[np.argmax(y_train[i])] + '\n')

Features: afterwards duke of york york earl of warwick warwick earl of salisbury salisbury earl of suffolk suffolk lord talbot

Label: afterwards

Features: of an ox his horses are bred better for besides that they are fair with their feeding they are

Label: taught



## Build The Model

Use `keras.Sequential` to define the model. For this simple example three layers are used to define our model:

* `keras.layers.Embedding`: The input layer. A trainable lookup table that will map the numbers of each character to a vector with `embedding_dim` dimensions;
* `keras.layers.LSTM`: A type of RNN with size `units=rnn_units` (You can also use a GRU layer here.)
* `keras.layers.Dense`: The output layer, with `vocab_size` outputs.

In [0]:
model = Sequential()

# Embedding layer
model.add(
    Embedding(
        input_dim=len(word_idx) + 1,
        output_dim=100,
        weights=None,
        trainable=True))

# Recurrent layer
model.add(
    LSTM(
        64, return_sequences=False, dropout=0.1,
        recurrent_dropout=0.1))

# Fully connected layer
model.add(Dense(64, activation='relu'))

# Dropout for regularization
model.add(Dropout(0.5))

# Output layer
model.add(Dense(num_words, activation='softmax'))

# Compile the model
model.compile(
    optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 100)         2694500   
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                42240     
_________________________________________________________________
dense_1 (Dense)              (None, 64)                4160      
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 26945)             1751425   
Total params: 4,492,325
Trainable params: 4,492,325
Non-trainable params: 0
___________

For each word the model looks up the embedding, runs the LSTM one timestep with the embedding as input, and applies the dense layer to generate logits predicting the log-liklihood of the next word.

## Train the model

In [0]:
h = model.fit(X_train, y_train, epochs = 500, batch_size = 50, 
          verbose = 1)## Train the model

Instructions for updating:
Use tf.cast instead.
Epoch 1/500
8820/8820 [==============================] - 16s 2ms/step - loss: 10.1995 - acc: 0.0053
Epoch 2/500
8820/8820 [==============================] - 14s 2ms/step - loss: 10.1889 - acc: 0.0261
Epoch 3/500
8820/8820 [==============================] - 14s 2ms/step - loss: 10.1661 - acc: 0.0290
Epoch 4/500
8820/8820 [==============================] - 14s 2ms/step - loss: 10.0813 - acc: 0.0293
Epoch 5/500
8820/8820 [==============================] - 13s 2ms/step - loss: 9.6923 - acc: 0.0288
Epoch 6/500
8820/8820 [==============================] - 14s 2ms/step - loss: 8.6819 - acc: 0.0312
Epoch 7/500
8820/8820 [==============================] - 13s 2ms/step - loss: 7.5724 - acc: 0.0330
Epoch 8/500
8820/8820 [==============================] - 14s 2ms/step - loss: 6.9266 - acc: 0.0332
Epoch 9/500
8820/8820 [==============================] - 14s 2ms/step - loss: 6.8929 - acc: 0.0322
Epoch 10/500
8820/8820 [==============================] -

### Save Model

In [0]:
# save the model to file
model.save('./data/model_1000epochs.h5')

In [0]:
pwd

'/content/drive/My Drive/NLP/text_generation/shakespeare'

### Load Model

In [0]:
# load the model
model = load_model('./data/model_1000epochs.h5')

### Note: After loading the model run  model.fit()  to continue training form there.

In [0]:
model.fit(X_train, y_train, batch_size=50, epochs=500)

Epoch 1/500
8820/8820 [==============================] - 24s 3ms/step - loss: 3.7882 - acc: 0.1862
Epoch 2/500
8820/8820 [==============================] - 24s 3ms/step - loss: 3.6207 - acc: 0.2069
Epoch 3/500
8820/8820 [==============================] - 24s 3ms/step - loss: 3.4319 - acc: 0.2333
Epoch 4/500
8820/8820 [==============================] - 24s 3ms/step - loss: 3.2801 - acc: 0.2520
Epoch 5/500
8820/8820 [==============================] - 24s 3ms/step - loss: 3.1446 - acc: 0.2706
Epoch 6/500
8820/8820 [==============================] - 24s 3ms/step - loss: 3.0767 - acc: 0.2766
Epoch 7/500
8820/8820 [==============================] - 24s 3ms/step - loss: 2.9792 - acc: 0.2875
Epoch 8/500
8820/8820 [==============================] - 24s 3ms/step - loss: 2.9446 - acc: 0.2949
Epoch 9/500
8820/8820 [==============================] - 24s 3ms/step - loss: 2.8640 - acc: 0.3078
Epoch 10/500
8820/8820 [==============================] - 24s 3ms/step - loss: 2.8072 - acc: 0.3159
Epoch 11/

## Evaluation

In [0]:
print(model.evaluate(X_train, y_train, batch_size = 20))
print('\nModel Performance: Log Loss and Accuracy on validation data')
print(model.evaluate(X_valid, y_valid, batch_size = 20))

8820/8820 [==============================] - 7s 792us/step
[0.017622768033935757, 0.9985260767460951]

Model Performance: Log Loss and Accuracy on validation data
3780/3780 [==============================] - 3s 808us/step
[13.280919408041333, 0.07857143015615524]


## Generate text

In [0]:
seed_length=50
new_words=50
diversity=1
n_gen=1

import random

# Choose a random sequence
seq = random.choice(sequences)

# print seq

# Choose a random starting point
seed_idx = random.randint(0, len(seq) - seed_length - 10)
# Ending index for seed
end_idx = seed_idx + seed_length

gen_list = []

for n in range(n_gen):
    # Extract the seed sequence
    seed = seq[seed_idx:end_idx]
    original_sequence = [idx_word[i] for i in seed]
    generated = seed[:] + ['#']

    # Find the actual entire sequence
    actual = generated[:] + seq[end_idx:end_idx + new_words]
        
    # Keep adding new words
    for i in range(new_words):

        # Make a prediction from the seed
        preds = model.predict(np.array(seed).reshape(1, -1))[0].astype(np.float64)

        # Diversify
        preds = np.log(preds) / diversity
        exp_preds = np.exp(preds)

        # Softmax
        preds = exp_preds / sum(exp_preds)

        # Choose the next word
        probas = np.random.multinomial(1, preds, 1)[0]

        next_idx = np.argmax(probas)

        # New seed adds on old word
        #             seed = seed[1:] + [next_idx]
        seed += [next_idx]
        generated.append(next_idx)
    # Showing generated and actual abstract
    n = []

    for i in generated:
        n.append(idx_word.get(i, '< --- >'))

    gen_list.append(n)

a = []

for i in actual:
    a.append(idx_word.get(i, '< --- >'))

a = a[seed_length:]

gen_list = [gen[seed_length:seed_length + len(a)] for gen in gen_list]

print('Original Sequence: \n'+' '.join(original_sequence))
print("\n")
# print(gen_list)
print('Generated Sequence: \n'+' '.join(gen_list[0][1:]))
# print(a)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:36: RuntimeWarning: divide by zero encountered in log


Original Sequence: 
me true though in his tale lie death i hear him as he flatter'd messenger labienus this is stiff news hath with his parthian force extended asia from euphrates his conquering banner shook from syria to lydia and to ionia whilst mark antony antony thou wouldst say messenger o my


Generated Sequence: 
lord a baggage the baggage the fairest in titus known messenger scene my gentleman to well no not a merchant here hear a passage 'tis whose unear'd art thou and sad me say not much ever he to make a noble sergeant coat slender i his brother at his blessing
